In [2]:
import os
import math
import random
import seaborn
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from keras.callbacks import EarlyStopping
from collections import namedtuple
from matplotlib.ticker import FormatStrFormatter

In [3]:
df = pd.read_excel("M3C.xls", usecols="A:Z")

df_micro = df.iloc[0:146,]
df_micro = df_micro.iloc[:,6:27]

In [ ]:
num_rows = 2
num_cols = 2

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))
axes = axes.flatten()
plots = [0, 4, 60, 98]
for i, ax in enumerate(axes):
    if i < len(df_micro):
        ax.plot(df_micro.iloc[plots[i]], marker="o")
        ax.set_title(f"Time-Series {plots[i]+1}")
        ax.set_xlabel("Time")
        ax.set_ylabel("Value")
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.set_xticks(range(1, 21))
    else:
        ax.axis("off")

plt.tight_layout()
plt.xticks(list(range(1, 21)))
plt.show()

In [4]:
df_train = df_micro.iloc[:,:-6]
df_test = df_micro.iloc[:, -6:]
df_train_original = df_train.to_numpy().reshape(-1, 14)

In [5]:
# Smoothing
#df_train_unsmoothed = df_train.to_numpy().reshape(-1,14)
def exponential_smoothing(data, alpha=0.8):
    return data.ewm(alpha=alpha, adjust=False).mean()

df_train = exponential_smoothing(df_train)

In [6]:
# Standardising
scaler = StandardScaler()
df_train = scaler.fit_transform(df_train.to_numpy().reshape(-1,1))
df_train = pd.DataFrame(df_train)
MEAN = scaler.mean_
STD = scaler.scale_

In [7]:
df_train = df_train.to_numpy().reshape(-1,14)
df_test = df_test.to_numpy().reshape(-1,6)

In [ ]:
import seaborn as sns

data = scaled_data

# Plotting the density plot
sns.kdeplot(data, shade=True, color='blue')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Density Plot of Data')
plt.show()

In [ ]:
df_train_unsmoothed = df_train
num_rows = 2
num_cols = 2

smoothed_graphs = [0, 4, 60, 98]
fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))
axes = axes.flatten()
counter = 0
for i, ax in enumerate(axes):
    if i < len(df_micro):
        ax.plot(df_train[smoothed_graphs[counter]], label="Pre-Processed Data", marker="o")
        ax.set_title(f"Time-Series {smoothed_graphs[counter] + 1}")
        ax.set_xlabel("Time")
        ax.set_ylabel("Value")
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.set_xticks(range(1, 14))
        ax.legend()
    else:
        ax.axis("off")
    counter = counter + 1

plt.tight_layout()
plt.xticks(list(range(1, 14)))
plt.show()

In [8]:
def get_labelled_window(x, horizon=1):
  return x[:, :-horizon], x[:, -horizon]

def make_windows(x, window_size=4, horizon=1):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of window size
  windowed_array = x[window_indexes]
  windows, labels = get_labelled_window(windowed_array, horizon=horizon)
  return windows.reshape(-1,4), labels.reshape(-1,1)

In [9]:
train_x = []
train_y = []
test_x = []
test_y = []

for i in range(len(df_train)):
    windows_train, labels_train = make_windows(df_train[i], window_size=4, horizon=1)
    windows_test, labels_test = make_windows(df_test[i], window_size=4, horizon=1)
    train_x = np.concatenate((np.array(train_x).reshape(-1,4), windows_train.reshape(-1,4)))
    train_y = np.concatenate((np.array(train_y).reshape(-1,1), labels_train.reshape(-1,1)))
    test_x = np.concatenate((np.array(test_x).reshape(-1,4), windows_test.reshape(-1,4)))
    test_y = np.concatenate((np.array(test_y).reshape(-1,1), labels_test.reshape(-1,1)))

In [10]:
# SMAPE
def evaluate_smape(y_true, y_pred):
    numerator = tf.abs(y_pred - y_true)
    denominator = (tf.abs(y_pred) + tf.abs(y_true))/2
    smape = tf.reduce_mean(numerator / denominator) * 100
    return smape

def smape_loss(y_true, y_pred):
    numerator = tf.abs(y_pred - y_true)
    denominator = ((tf.abs(y_pred) + tf.abs(y_true))/2) + 0.1
    smape = tf.reduce_mean(numerator / denominator)
    return smape

def metric_mdape(y_true, y_pred):
    return tfp.stats.percentile((tf.abs(tf.math.subtract(y_true, y_pred)/ y_true)), 50.0, interpolation='midpoint')
      
def evaluate_mdape(y_true, y_pred):
    return np.median((np.abs(np.subtract(y_true, y_pred)/ y_true))) * 100

def evaluate_pred(y_true, y_pred):
    # Symmetric mean absolute percentage error
    smape = evaluate_smape(y_true, y_pred)
    # Median symmetric absolute percentage error
    mdape = evaluate_mdape(y_true, y_pred)
    return smape, mdape

In [11]:
# Destandardise
def de_standardise(value):
    return value * STD + MEAN

def standardise(value):
    return (value - MEAN) / STD

In [12]:
# Hyperparameters
Combination = namedtuple("Combination", "learning_rate batch_size regularization hidden_layers hidden_neurons")

learning_rates = np.array([0.001, 0.01, 0.1])
batch_sizes = np.array([16, 32, 64, 128])
regularizations = np.array([0.001, 0.01, 0.001])
hidden_layers = np.array([2, 3, 4, 5, 6, 8]) # 2, 3, 4, 6, 8
hidden_neurons = np.array([2, 3, 4, 5])

combinations = list(itertools.starmap(Combination, itertools.product(learning_rates, batch_sizes, regularizations, hidden_layers, hidden_neurons)))

In [ ]:
# Time-series expanding window validation
#with tf.device('/cpu:0'):
    
tf.random.set_seed(42)
eval_scores = []
tscv = TimeSeriesSplit(n_splits=5)
def cross_validation(combination, train_x=train_x, train_y=train_y, tscv=tscv):
    smape_scores = []
    mdape_scores = []

    # Cross-Validation
    for train_index, test_index in tscv.split(train_x):
        train_x_cv, val_x_cv = train_x[train_index], train_x[test_index]
        train_y_cv, val_y_cv = train_y[train_index], train_y[test_index]
        # Create model with selected hyperparameters
        model_cv = tf.keras.Sequential(name="model")

        for i in range(combination.hidden_layers):
            model_cv.add(tf.keras.layers.Dense(combination.hidden_neurons, 
                                            activation="relu", 
                                            kernel_initializer=tf.initializers.HeNormal(), 
                                            kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))
        model_cv.add(tf.keras.layers.Dense(1, activation="linear", 
                                        kernel_initializer=tf.initializers.HeNormal(), 
                                        kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))


        model_cv.compile(loss=smape_loss,
                        optimizer=tf.keras.optimizers.Adam(learning_rate=combination.learning_rate),
                        metrics=[metric_mdape, "mae", "mse"]) # Backpropagation
        
        model_cv.fit(train_x_cv, train_y_cv, epochs=50, batch_size=combination.batch_size, verbose=0)

        predictions = model_cv.predict(val_x_cv)

        smape_score, mdape_score = evaluate_pred(de_standardise(val_y_cv), de_standardise(predictions))
        
        smape_scores.append(smape_score)
        mdape_scores.append(mdape_score)
        
    mean_smape = np.mean(smape_scores)
    mean_mdape = np.mean(mdape_scores)
    hyperparameters = {
        'learning_rate': combination.learning_rate,
        'batch_size': combination.batch_size,
        'regularization': combination.regularization,
        'hidden_neurons': combination.hidden_neurons,
        'hidden_layers': combination.hidden_layers
    }
    print(f"Current mean SMAPE: {mean_smape}, Current hyperparameters: {hyperparameters}")
    return mean_smape, mean_mdape, hyperparameters

random_combinations = random.sample(combinations, 20)
results = map(cross_validation, random_combinations)

optimal_smape = float('inf')
optimal_mdape = float('inf')
optimal_hyperparameters = {}
for result in results:
    smape, mdape, hyperparameters = result
    if smape < optimal_smape:
        optimal_smape = smape
        optimal_mdape = mdape
        optimal_hyperparameters = hyperparameters
print("Best Hyperparameters:", optimal_hyperparameters)
print("Best SMAPE Score:", optimal_smape)
print("Best MDAPE Score:", optimal_mdape)

In [ ]:
print(f"Regularization: {optimal_hyperparameters['regularization']}")
print(f"Learning Rate: {optimal_hyperparameters['learning_rate']}")
print(f"Batch Size: {optimal_hyperparameters['batch_size']}")
print(f"Hidden Layers: {optimal_hyperparameters['hidden_layers']}")
print(f"Hidden Neurons: {optimal_hyperparameters['hidden_neurons']}")

model = tf.keras.Sequential([
    #tf.keras.layers.Flatten(input_shape=(4, 1)),
], name="model")

for i in range(optimal_hyperparameters["hidden_layers"]):
    model.add(tf.keras.layers.Dense(optimal_hyperparameters["hidden_neurons"], 
                                    activation="relu", 
                                    kernel_initializer=tf.initializers.HeNormal(), 
                                    kernel_regularizer=tf.keras.regularizers.l2(optimal_hyperparameters["regularization"])))
model.add(tf.keras.layers.Dense(1, activation="linear", 
                                kernel_initializer=tf.initializers.HeNormal(), 
                                kernel_regularizer=tf.keras.regularizers.l2(optimal_hyperparameters["regularization"])))

print()
model.compile(loss=smape_loss,
                optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_hyperparameters["learning_rate"]), 
                metrics=[metric_mdape, "mae", "mse"]) # Backpropagation

early_stopping = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=50)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint('best_model_5.hdf5', monitor='loss', mode='min', verbose=1, save_best_only=True)
# Train the model on the full training dataset
model.fit(train_x, train_y, epochs=500, batch_size=optimal_hyperparameters["batch_size"], verbose=1, callbacks=[early_stopping, model_checkpoint])

In [ ]:
model.summary()

In [13]:
def autoregression(model, x, horizon=6):
    standardised_x = standardise(x)
    for i in range(horizon):
        forecast = model.predict(np.array([standardised_x[i:i+4]]))
        pred = np.array([tf.squeeze(forecast).numpy()])
        standardised_x = np.concatenate((standardised_x, pred))
    return standardised_x[-horizon:]

In [14]:
predictions = []
def evaluate_model_on_test(model, df_train=df_train, df_test=df_test, horizon=1):
    smape_scores = []
    mdape_scores = []
    
    for i in range(len(df_train)):
        window = de_standardise(df_train[i][10:14])
        labels = df_test[i][0:horizon]
        test_preds = autoregression(model, window, horizon)
        #print(f"Destandardised test pred: {de_standardise(test_preds)}")
        
        #print(f"Full Labels: {df_test[i]}")
        #print(f"Window: {window}")
        predictions.append(de_standardise(test_preds))
        print(f"Labels: {labels}, Window: {window}")
        print(f"Label: {labels}, Pred: {de_standardise(test_preds)}")
        smape_score, mdape_score = evaluate_pred(labels, de_standardise(test_preds))
        smape_scores.append(smape_score)
        mdape_scores.append(mdape_score)
        print(f"Current mean SMAPE: {smape_score}, Current mean MDAPE: {mdape_score}")

    mean_smape_score = np.mean(smape_scores)
    mean_mdape_score = np.mean(mdape_scores)
    return mean_smape_score, mean_mdape_score


In [ ]:
loaded_model = tf.keras.models.load_model("best_model_4.hdf5", custom_objects={"smape_loss": smape_loss, "metric_mdape": metric_mdape})
test1, test2 = evaluate_model_on_test(loaded_model, horizon=6)

In [ ]:
test1

In [ ]:
num_rows = 2
num_cols = 2

fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 8))
axes = axes.flatten()
plots = [1, 7, 99, 134]
for i, ax in enumerate(axes):
    if i < len(df_micro):
        testplt = np.append(df_micro.iloc[plots[i]][:14], predictions[plots[i]])
        ax.plot(list(range(1, 21)), testplt, marker='o', label='Predicted')
        ax.plot(list(range(1, 21)), df_micro.iloc[plots[i]].to_numpy(), marker='o', label="Actual")
        ax.axvline(x = 14, color = 'gray', linestyle='--')
        ax.set_title(f"Time-Series {plots[i]+1} - Original Data vs. Predicted Data")
        ax.xaxis.set_major_formatter(FormatStrFormatter('%.0f'))
        ax.set_xticks(range(1, 21))
        ax.set_xlabel("Time")
        ax.set_ylabel("Value")
        ax.legend()
    else:
        ax.axis("off")

plt.tight_layout()
plt.xticks(list(range(1, 21)))
plt.show()

In [ ]:
def plot_pred(index):
    testplt = np.append(df_micro.iloc[index][:14], predictions[index])
    plt.plot(list(range(1, 21)), testplt, marker='o', label='Predicted')
    plt.plot(list(range(1, 21)), df_micro.iloc[index].to_numpy(), marker='o', label="Actual")
    plt.axvline(x = 14, color = 'gray', linestyle='--')
    plt.title(f"Time-Series {index+1} - Original Data vs. Predicted Data")
    plt.xticks(list(range(1, 21)))
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.legend()
    plt.show()
plot_pred(98)